# Rob's Capstone Project

This notebook with contain my work for the [capstone project](https://www.coursera.org/learn/applied-data-science-capstone/home/welcome) for IBM Data Science Specialization certificate.
### Week 4

**Name**: Robert Barrimond

**Date**: May 15, 2021

**REVIEWER PLEASE NOTE**
I do _not_ comment my code with Markdown. As an SRE (Site Reliability Engineer), I do as application developers should do: document code _in the code_ and everywhere possible by the code itself. Having said that, SREs are also called to be data scientists as well. So, I use Markdown to "tell the story" as that first overview course taught me so many months ago. I hope this assignment was easy to follow and grade!

---

## Problem Statement
I've decided to see if it's worth pursuing opening a Cambodian restaurant somewhere in Toronto. I noticed from the previous assignments that the city is quite cosmopolitan and would welcome such a restaurant. The real problem is _where_ to locate it. My strategy will be to narrow the list of venues to Asian restaurants, get premium data on just those venues, e.g. number of like and rating, and use that to produce better clusters that can make my decision easier.


In [1]:
#
# Import the necessary modules
#

# Data analysis and transformation
import pandas as pd
import numpy as np

# REST API access
import requests

# File access
import os
from os import path
import pickle

# Geocoders
import geopy
from geopy.geocoders import Nominatim

# Regex
import re

## Retrieve and Clean Data for Analysis
### Build List of Asian Restaurants using the Foursquare API

Because it took [some work](https://github.com/rbarrimond/Coursera_Capstone/blob/5afe9c180839529c96ee71c8a5fae69746b9f4c3/toronto-kmeans-clustering.ipynb) to build a clean dataframe of FSAs from Wikipedia, I'll omit that work here and simple read the pickle from disk. Next I use the Foursquare API `explore` endpoint to get all the nearby venues. Then I'll explore the clean dataframe of Toronto venues to see I know that the classifications from the Week 4 assignment were too broad. My first task is to see if I can narrow down the venues to Asian restaurants from a list of almost 270 categories. I guessed that categories ending in 'restaurant', 'joint', and 'place' would be enough to cover it. I managed to find 145 Asian restaurants.

In [2]:
# Load the FSA data from previous work
fsa_df = pd.read_pickle('fsa_df.pkl')
fsa_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [3]:
# FourSquare credentials
CLIENT_ID = '0II4MXQK5GVKQA3YIZRXT3D0KWBAKEH2BCCYRWIK4H0DS5XH' # your Foursquare ID
CLIENT_SECRET = 'ZBOCOGUCP2AXOQNAFSPX05IAXWAPBNUBC2FTAGYJV4DDS3AA' # your Foursquare Secret
ACCESS_TOKEN = 'ESEQDUIWNVRAS11OKDDXMICGNUPXLZCHPVHCZ53OTT2LQWBS' # your FourSquare Access Token
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [4]:
# This function takes a sequence of names, lats and longs and produces a dataframe of nearby venues
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['id'],
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name'] ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue ID',
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [5]:
# Retrieve all the venues in Toronto
toronto_venues = getNearbyVenues(fsa_df['PostalCode'], fsa_df['Latitude'], fsa_df['Longitude'])
toronto_venues.head(10)

M3A
M4A
M5A
M6A
M7A
M9A
M1B
M3B
M4B
M5B
M6B
M9B
M1C
M3C
M4C
M5C
M6C
M9C
M1E
M4E
M5E
M6E
M1G
M4G
M5G
M6G
M1H
M2H
M3H
M4H
M5H
M6H
M1J
M2J
M3J
M4J
M5J
M6J
M1K
M2K
M3K
M4K
M5K
M6K
M1L
M2L
M3L
M4L
M5L
M6L
M9L
M1M
M2M
M3M
M4M
M5M
M6M
M9M
M1N
M2N
M3N
M4N
M5N
M6N
M9N
M1P
M2P
M4P
M5P
M6P
M9P
M1R
M2R
M4R
M5R
M6R
M7R
M9R
M1S
M4S
M5S
M6S
M1T
M4T
M5T
M1V
M4V
M5V
M8V
M9V
M1W
M4W
M5W
M8W
M9W
M1X
M4X
M5X
M8X
M4Y
M7Y
M8Y
M8Z


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,4e8d9dcdd5fbbbb6b3003c7b,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,4cb11e2075ebb60cd1c4caad,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,4c633acb86b6be9a61268e34,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Portugril,4f3ecce6e4b0587016b6f30d,43.725819,-79.312785,Portuguese Restaurant
4,M4A,43.725882,-79.315572,Tim Hortons,4bbe904a85fbb713420d7167,43.725517,-79.313103,Coffee Shop
5,M4A,43.725882,-79.315572,Eglinton Ave E & Sloane Ave/Bermondsey Rd,51e594d5498e07837484f577,43.726086,-79.313620,Intersection
6,M4A,43.725882,-79.315572,Pizza Nova,4b8fe578f964a520106833e3,43.725824,-79.312860,Pizza Place
7,M5A,43.654260,-79.360636,Roselle Desserts,54ea41ad498e9a11e9e13308,43.653447,-79.362017,Bakery
8,M5A,43.654260,-79.360636,Tandem Coffee,53b8466a498e83df908c3f21,43.653559,-79.361809,Coffee Shop
9,M5A,43.654260,-79.360636,Cooper Koo Family YMCA,574c229e498ebb5c6b257902,43.653249,-79.358008,Distribution Center


In [6]:
# Explore the types of restaurants
restaurants = toronto_venues.loc[ toronto_venues['Venue Category'].str.contains('restaurant', case=False, regex=True) ].copy()
types = restaurants['Venue Category'].str.replace("Restaurant", "")
types = pd.Series(types.unique())
types = types.sort_values().reset_index(drop=True)
types

0                          
1                 American 
2                    Asian 
3                  Belgian 
4                Brazilian 
5           Cajun / Creole 
6                Caribbean 
7                  Chinese 
8                Colombian 
9             Comfort Food 
10                   Cuban 
11                 Dim Sum 
12                   Doner 
13                Dumpling 
14        Eastern European 
15               Ethiopian 
16                 Falafel 
17               Fast Food 
18                Filipino 
19                  French 
20                  German 
21             Gluten-free 
22                   Greek 
23                   Hakka 
24                  Indian 
25                 Italian 
26                Japanese 
27                  Korean 
28              Korean BBQ 
29          Latin American 
30           Mediterranean 
31                 Mexican 
32          Middle Eastern 
33         Modern European 
34    Molecular Gastronomy 
35                Mo

In [7]:
# Explore the types of joints
joints = toronto_venues.loc[ toronto_venues['Venue Category'].str.contains('joint', case=False) ]
types = joints['Venue Category'].str.replace("Joint", "")
types = pd.Series(types.unique())
types = types.sort_values().reset_index(drop=True)
types

0              BBQ 
1           Burger 
2    Fried Chicken 
3            Wings 
dtype: object

In [8]:
# Explore the types of places
places = toronto_venues.loc[ toronto_venues['Venue Category'].str.contains('place', case=False) ]
types = places['Venue Category'].str.replace("Place", "")
types = pd.Series(types.unique())
types = types.sort_values().reset_index(drop=True)
types

0     Burrito 
1       Pizza 
2        Poke 
3     Poutine 
4       Salad 
5    Sandwich 
6       Snack 
7        Soup 
8        Taco 
dtype: object

In [9]:
# Check the bars just to be sure I'm not missing anything Asian themed
bars = toronto_venues.loc[ toronto_venues['Venue Category'].str.contains('bar$', case=False, regex=True) ]
types = bars['Venue Category'].str.replace("Bar$", "", regex=True)
types = pd.Series(types.unique())
types = types.sort_values().reset_index(drop=True)
types

0             
1        Beer 
2    Cocktail 
3         Gay 
4      Hookah 
5       Hotel 
6       Juice 
7        Sake 
8      Sports 
9        Wine 
dtype: object

In [10]:
# Set the Asian categories based on the above and create a cleaned Asian restaurants dataframe
asian_categories = [ 'Asian', 'Chinese', 'Dim Sum', 'Dumpling', 'Filipino', 'Japanese', 'Korean', 'Korean BBQ', 'Ramen', 'Sushi', 'Taiwanese', 'Thai', 'Vietnamese']
asian_restaurants = toronto_venues.loc[ toronto_venues['Venue Category'].str.contains("|".join(asian_categories) + ' restaurant', case=False, regex=True) ].copy()
asian_restaurants.reset_index(drop=True, inplace=True)
asian_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category
0,M5A,43.654260,-79.360636,Izumi,50e88fc6e4b007fcbb57aae8,43.649970,-79.360153,Asian Restaurant
1,M6A,43.718518,-79.464763,Lac Vien Vietnamese Restaurant,4ccc5aebee23a14370591ea8,43.721259,-79.468472,Vietnamese Restaurant
2,M7A,43.662301,-79.389494,Tokyo Sushi,4afca145f964a5208e2422e3,43.665885,-79.386977,Sushi Restaurant
3,M7A,43.662301,-79.389494,Sushi Garden,4ba6b463f964a520416a39e3,43.664858,-79.384941,Sushi Restaurant
4,M7A,43.662301,-79.389494,Tokyo Grill,4ad4c061f964a520abf720e3,43.665085,-79.384707,Japanese Restaurant
...,...,...,...,...,...,...,...,...
140,M4Y,43.665860,-79.383160,Tokyo Kitchen,4ae787b9f964a52020ac21e3,43.668783,-79.385153,Japanese Restaurant
141,M4Y,43.665860,-79.383160,Onnki Donburi,59c54d4f2d2fd97564d4cfc8,43.669757,-79.384574,Japanese Restaurant
142,M4Y,43.665860,-79.383160,Miss Korea,527428a5498e8e23580dbe15,43.668950,-79.386165,Korean Restaurant
143,M4Y,43.665860,-79.383160,Kokoni Izakaya,4c531b60a724e21e029e3af4,43.664181,-79.380258,Japanese Restaurant


### Pull Full Data from Foursquare for Each Asian Restaurant
NOTE: The `/venue` endpoint is a premium API. As a result, I cached results to file. 


In [11]:
# Read in venue data either from cache or from Foursquare
asian_venue_data = {}
if path.exists('asian_venue_data.pkl'):
    with open('asian_venue_data.pkl', 'rb') as f:
        asian_venue_data = pickle.load(f)
else:
    for venue_id in asian_restaurants['Venue ID']:
        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
            venue_id, CLIENT_ID, CLIENT_SECRET,ACCESS_TOKEN, VERSION)
        r = requests.get(url)
        if r.status_code == 200:
            asian_venue_data[venue_id] = r.json()['response']
            print('Read', asian_venue_data[venue_id]['venue']['name'])
        else:
            print('Unable to retrieve data for ID:', venue_id)
    with open('asian_venue_data.pkl', 'wb') as f:
        pickle.dump(asian_venue_data, f, pickle.HIGHEST_PROTOCOL)
print("Pulled data for", len(asian_venue_data), "venues.")

Pulled data for 119 venues.


In [12]:
# Create a dataframe for the venue data
asian_venue_data_df = pd.DataFrame()
for id in asian_venue_data:
    asian_venue_data_df = asian_venue_data_df.append(pd.json_normalize(asian_venue_data[id]), ignore_index=True)
asian_venue_data_df.set_index('venue.id', inplace=True)
asian_venue_data_df

,venue.name,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.canonicalUrl,...,venue.parent.location.labeledLatLngs,venue.parent.location.postalCode,venue.parent.location.cc,venue.parent.location.city,venue.parent.location.state,venue.parent.location.country,venue.parent.location.formattedAddress,venue.parent.categories,venue.hierarchy,venue.parent.location.neighborhood
venue.id,,,,,,,,,,,,,,,,,,,,,
50e88fc6e4b007fcbb57aae8,Izumi,43.649970,-79.360153,"[{'label': 'display', 'lat': 43.6499697935016,...",CA,Toronto,ON,Canada,"[Toronto ON, Canada]",https://foursquare.com/v/izumi/50e88fc6e4b007f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4ccc5aebee23a14370591ea8,Lac Vien Vietnamese Restaurant,43.721259,-79.468472,"[{'label': 'display', 'lat': 43.72125878799614...",CA,Toronto,ON,Canada,"[141 Cartwright Ave, Toronto ON, Canada]",https://foursquare.com/v/lac-vien-vietnamese-r...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4afca145f964a5208e2422e3,Tokyo Sushi,43.665885,-79.386977,"[{'label': 'display', 'lat': 43.66588533093684...",CA,Toronto,ON,Canada,"[33 St. Joseph St. (Yonge and Wellesley), Toro...",https://foursquare.com/v/tokyo-sushi/4afca145f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4ba6b463f964a520416a39e3,Sushi Garden,43.664858,-79.384941,"[{'label': 'display', 'lat': 43.66485840073255...",CA,Toronto,ON,Canada,"[3 Wellesley St W (at Yonge), Toronto ON, Canada]",https://foursquare.com/v/sushi-garden/4ba6b463...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4ad4c061f964a520abf720e3,Tokyo Grill,43.665085,-79.384707,"[{'label': 'display', 'lat': 43.66508523814939...",CA,Toronto,ON,Canada,"[582 Yonge St (at Wellesley St W), Toronto ON ...",https://foursquare.com/v/tokyo-grill/4ad4c061f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4ae787b9f964a52020ac21e3,Tokyo Kitchen,43.668783,-79.385153,"[{'label': 'display', 'lat': 43.66878262373862...",CA,Toronto,ON,Canada,"[Charles St. (at Yonge St.), Toronto ON, Canada]",https://foursquare.com/v/tokyo-kitchen/4ae787b...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59c54d4f2d2fd97564d4cfc8,Onnki Donburi,43.669757,-79.384574,"[{'label': 'display', 'lat': 43.66975742655123...",CA,Toronto,ON,Canada,"[40 Hayden Street, Toronto ON M4Y 1V8, Canada]",https://foursquare.com/v/onnki-donburi/59c54d4...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
527428a5498e8e23580dbe15,Miss Korea,43.668950,-79.386165,"[{'label': 'display', 'lat': 43.66894989012164...",CA,Toronto,ON,Canada,"[687 Yonge St (at Charles St), Toronto ON M6H ...",https://foursquare.com/v/miss-korea/527428a549...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#
# Join key data fields to the restaurants data frame and identify the dupes
#
key_venue_cols = [
    'venue.location.address',
    'venue.location.postalCode',
    'venue.stats.tipCount',
    'venue.price.tier',
    'venue.rating',
    'venue.likes.count'
]
asian_restaurants = asian_restaurants.join(asian_venue_data_df[key_venue_cols], on='Venue ID')

# Cleanup
asian_restaurants.rename(columns={
    'venue.location.address': 'Venue Address',
    'venue.location.postalCode': 'Venue Postal Code',
    'venue.stats.tipCount': 'Venue Tip Count',
    'venue.price.tier': 'Venue Price Tier',
    'venue.rating': 'Venue Rating',
    'venue.likes.count': 'Venue Likes'
}, inplace=True)
asian_restaurants['Venue Price Tier'] = asian_restaurants['Venue Price Tier'].astype(pd.Int64Dtype())
asian_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Address,Venue Postal Code,Venue Tip Count,Venue Price Tier,Venue Rating,Venue Likes
0,M5A,43.654260,-79.360636,Izumi,50e88fc6e4b007fcbb57aae8,43.649970,-79.360153,Asian Restaurant,NaN,NaN,4,2,7.4,18
1,M6A,43.718518,-79.464763,Lac Vien Vietnamese Restaurant,4ccc5aebee23a14370591ea8,43.721259,-79.468472,Vietnamese Restaurant,141 Cartwright Ave,NaN,12,2,7.6,29
2,M7A,43.662301,-79.389494,Tokyo Sushi,4afca145f964a5208e2422e3,43.665885,-79.386977,Sushi Restaurant,33 St. Joseph St.,M4Y 1J8,24,2,8.1,32
3,M7A,43.662301,-79.389494,Sushi Garden,4ba6b463f964a520416a39e3,43.664858,-79.384941,Sushi Restaurant,3 Wellesley St W,NaN,9,2,7.7,9
4,M7A,43.662301,-79.389494,Tokyo Grill,4ad4c061f964a520abf720e3,43.665085,-79.384707,Japanese Restaurant,582 Yonge St,M4Y 1Z3,29,1,7.7,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,M4Y,43.665860,-79.383160,Tokyo Kitchen,4ae787b9f964a52020ac21e3,43.668783,-79.385153,Japanese Restaurant,Charles St.,NaN,21,2,7.4,20
141,M4Y,43.665860,-79.383160,Onnki Donburi,59c54d4f2d2fd97564d4cfc8,43.669757,-79.384574,Japanese Restaurant,40 Hayden Street,M4Y 1V8,3,2,7.7,13
142,M4Y,43.665860,-79.383160,Miss Korea,527428a5498e8e23580dbe15,43.668950,-79.386165,Korean Restaurant,687 Yonge St,M6H 2C9,17,2,7.5,32
143,M4Y,43.665860,-79.383160,Kokoni Izakaya,4c531b60a724e21e029e3af4,43.664181,-79.380258,Japanese Restaurant,459 Church St,M4Y 2C5,9,2,6.6,1


#### The Problem of Duplicates
Initially I was surprised to find only 119 venues for 145 venues in the original dataframe. This means the dataframe above has duplicates. However, on further inspection, it seems reasonable since we didn't explore by _FSA_ rather by _distance from the FSA centroids_. This means a restaurant could (and actually did) show up under multiple FSAs. This begged the question: _To which FSA should each venue be assigned?_ I went with the officially assigned FSA for simplicity. Nominatum was very handy for filling in the blanks in the Foursquare data by leveraging the `Venue Address` to do geocode lookups. As a last resort, I did a reverse lookup on the `Venue Latitude` and `Venue Longitude` to find the postal code. (Unfortunately the exact addresses didn't seem to match the venues so I ignored them.)

In [14]:
# 
# Create masks to filter the dataframe
#

postalcode_mask = pd.notna(asian_restaurants['Venue Postal Code'])
postcode_match_mask = (asian_restaurants['Venue Postal Code'].str.extract('^(\w{3})', expand=False) == asian_restaurants['Neighborhood'])
address_mask = pd.notna(asian_restaurants['Venue Address'])

# Venues with mismatched postal codes
pd.set_option('max_rows', 200)
asian_restaurants[['Venue','Neighborhood','Venue Postal Code', 'Venue Address']].loc[~postcode_match_mask]

,Venue,Neighborhood,Venue Postal Code,Venue Address
0,Izumi,M5A,NaN,NaN
1,Lac Vien Vietnamese Restaurant,M6A,NaN,141 Cartwright Ave
2,Tokyo Sushi,M7A,M4Y 1J8,33 St. Joseph St.
3,Sushi Garden,M7A,NaN,3 Wellesley St W
4,Tokyo Grill,M7A,M4Y 1Z3,582 Yonge St
10,GB Hand-Pulled Noodles,M5B,M5G 1C9,66 Edward Street
12,KAKA,M5B,M5G 1Z4,655 Bay Street
14,Miyako Sushi Restaurant,M6B,NaN,572 Marlee Ave
15,Li Cheng Restaurant,M6B,NaN,529 Marlee Avenue
16,Asian Legend,M3C,NaN,NaN


As we can see there are a lot of mismatches in the data. The FSAs that we initially set as `Neighborhood` don't match the offical FSA in `Postal Code`. The good news is that most of them were pretty close so that bodes well for me to start adjusting the `Neighborhood` column to reflect the "true" FSA. The first thing to do is adjust the known FSAs.

In [15]:
# Adjust Neighborhood to known FSA
mask = postalcode_mask & ~postcode_match_mask
asian_restaurants.loc[ mask, 'Neighborhood'] = asian_restaurants.loc[mask, 'Venue Postal Code'].str.extract('^(\w{3})', expand=False)

# Reset masks
postalcode_mask = pd.notna(asian_restaurants['Venue Postal Code'])
postcode_match_mask = (asian_restaurants['Venue Postal Code'].str.extract('^(\w{3})', expand=False) == asian_restaurants['Neighborhood'])
address_mask = pd.notna(asian_restaurants['Venue Address'])

# Check results
asian_restaurants[['Venue','Neighborhood','Venue Postal Code', 'Venue Address']].loc[~postcode_match_mask]

,Venue,Neighborhood,Venue Postal Code,Venue Address
0,Izumi,M5A,NaN,NaN
1,Lac Vien Vietnamese Restaurant,M6A,NaN,141 Cartwright Ave
3,Sushi Garden,M7A,NaN,3 Wellesley St W
14,Miyako Sushi Restaurant,M6B,NaN,572 Marlee Ave
15,Li Cheng Restaurant,M6B,NaN,529 Marlee Avenue
16,Asian Legend,M3C,NaN,NaN
18,Pho 88 發發餐廳,M3C,NaN,900 Don Mills Road
19,Bento Sushi,M3C,NaN,NaN
22,Sweet Lulu,M5C,NaN,350 Bay St
25,Korean Grill House,M1G,NaN,369 Yonge Street


Now we have venues with no postal code but we _do_ have an address. Time to engage Nominatum to see if the geocoder can get us the postal code.

In [16]:
# Use the Nominatum geocoder to get the postal code data
geolocator = Nominatim(user_agent="robs_ba_explorer")
mask = ~postalcode_mask & ~postcode_match_mask & address_mask
postal_codes = pd.Series(dtype='object')
for address in asian_restaurants.loc[ mask, 'Venue Address']:
    location = geolocator.geocode(address + ", Toronto, ON")
    postal_codes[address] = location.address
postal_codes = postal_codes.str.extract("Ontario, (\w{3}(?:\s{1}\w{3})?).*$")   
postal_codes

,0
141 Cartwright Ave,M6A 2X3
3 Wellesley St W,M4Y 1E7
572 Marlee Ave,M6B 2A2
529 Marlee Avenue,M6B 2A2
900 Don Mills Road,M3C 2H2
350 Bay St,M5H
369 Yonge Street,M5B 1R8
100 Simcoe St.,M5H
Fairview Mall,M2J 5A7
1300 Finch Ave West,M3J 3K2


In [17]:
# Loop through and set the Postal Code 
for index in asian_restaurants.loc[mask].index:
    asian_restaurants.loc[index, 'Venue Postal Code'] = postal_codes.loc[asian_restaurants.loc[index, 'Venue Address'], 0]

# Reset masks
postalcode_mask = pd.notna(asian_restaurants['Venue Postal Code'])
postcode_match_mask = (asian_restaurants['Venue Postal Code'].str.extract('^(\w{3})', expand=False) == asian_restaurants['Neighborhood'])
address_mask = pd.notna(asian_restaurants['Venue Address'])
mask = ~postcode_match_mask & address_mask

# Check results
asian_restaurants[['Venue','Neighborhood','Venue Postal Code', 'Venue Address']].loc[mask]

,Venue,Neighborhood,Venue Postal Code,Venue Address
3,Sushi Garden,M7A,M4Y 1E7,3 Wellesley St W
22,Sweet Lulu,M5C,M5H,350 Bay St
25,Korean Grill House,M1G,M5B 1R8,369 Yonge Street
55,Bazara,M6J,M6G 1M4,188 Ossington
63,Ninki Izakaya,M5K,M5H,133 Richmond Street West
64,Sweet Lulu,M5K,M5H,350 Bay St
66,Szechuan Express,M5K,M5J 2S1,200 Bay St.
68,O Sushi,M4L,M4C 3J6,6 Coxwell
72,Sweet Lulu,M5L,M5H,350 Bay St
110,Daeco Sushi,M4V,M6C 1A1,40 St. Clair West


In [18]:
# Update the Neighborhood to match the FSA in the Postal Code
asian_restaurants.loc[ mask, 'Neighborhood'] = asian_restaurants.loc[mask, 'Venue Postal Code'].str.extract('^(\w{3})', expand=False)
asian_restaurants[['Venue','Neighborhood','Venue Postal Code', 'Venue Address']].loc[mask]

,Venue,Neighborhood,Venue Postal Code,Venue Address
3,Sushi Garden,M4Y,M4Y 1E7,3 Wellesley St W
22,Sweet Lulu,M5H,M5H,350 Bay St
25,Korean Grill House,M5B,M5B 1R8,369 Yonge Street
55,Bazara,M6G,M6G 1M4,188 Ossington
63,Ninki Izakaya,M5H,M5H,133 Richmond Street West
64,Sweet Lulu,M5H,M5H,350 Bay St
66,Szechuan Express,M5J,M5J 2S1,200 Bay St.
68,O Sushi,M4C,M4C 3J6,6 Coxwell
72,Sweet Lulu,M5H,M5H,350 Bay St
110,Daeco Sushi,M6C,M6C 1A1,40 St. Clair West


In [19]:
# Find remaining venues that need to be adjusted
mask = ~postcode_match_mask & ~postalcode_mask
asian_restaurants.loc[~postcode_match_mask & ~postalcode_mask]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Address,Venue Postal Code,Venue Tip Count,Venue Price Tier,Venue Rating,Venue Likes
0,M5A,43.654260,-79.360636,Izumi,50e88fc6e4b007fcbb57aae8,43.649970,-79.360153,Asian Restaurant,NaN,NaN,4,2,7.4,18
16,M3C,43.725900,-79.340923,Asian Legend,5686b77338faf7478eb6c6aa,43.726591,-79.342188,Dim Sum Restaurant,NaN,NaN,4,2,6.5,13
19,M3C,43.725900,-79.340923,Bento Sushi,4d6be9dd1013236abc9cb5f9,43.722265,-79.337631,Sushi Restaurant,NaN,NaN,1,2,NaN,0
42,M5H,43.650571,-79.384568,Ruby Thai (First Canadian Place),5632426a498e0433cad5aa04,43.649091,-79.381600,Thai Restaurant,NaN,NaN,2,2,7.1,7
73,M5L,43.648198,-79.379817,Ruby Thai (First Canadian Place),5632426a498e0433cad5aa04,43.649091,-79.381600,Thai Restaurant,NaN,NaN,2,2,7.1,7
131,M5X,43.648429,-79.382280,Ruby Thai (First Canadian Place),5632426a498e0433cad5aa04,43.649091,-79.381600,Thai Restaurant,NaN,NaN,2,2,7.1,7


In [20]:
# Do a reverse geocode lookup and extract postal code from address found
p = re.compile(r"Ontario, (\w{3}(?:\s{1}\w{3})?).*$")
for index in asian_restaurants[mask].index:
    location = geolocator.reverse("{}, {}".format(asian_restaurants.loc[index, 'Venue Latitude'], asian_restaurants.loc[index, 'Venue Longitude']))
    m = p.search(location.address)
    asian_restaurants.loc[index, 'Venue Postal Code'] = m.group(1)
    asian_restaurants.loc[index, 'Neighborhood'] = m.group(1)[0:3]
asian_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Address,Venue Postal Code,Venue Tip Count,Venue Price Tier,Venue Rating,Venue Likes
0,M5A,43.654260,-79.360636,Izumi,50e88fc6e4b007fcbb57aae8,43.649970,-79.360153,Asian Restaurant,NaN,M5A 3C4,4,2,7.4,18
1,M6A,43.718518,-79.464763,Lac Vien Vietnamese Restaurant,4ccc5aebee23a14370591ea8,43.721259,-79.468472,Vietnamese Restaurant,141 Cartwright Ave,M6A 2X3,12,2,7.6,29
2,M4Y,43.662301,-79.389494,Tokyo Sushi,4afca145f964a5208e2422e3,43.665885,-79.386977,Sushi Restaurant,33 St. Joseph St.,M4Y 1J8,24,2,8.1,32
3,M4Y,43.662301,-79.389494,Sushi Garden,4ba6b463f964a520416a39e3,43.664858,-79.384941,Sushi Restaurant,3 Wellesley St W,M4Y 1E7,9,2,7.7,9
4,M4Y,43.662301,-79.389494,Tokyo Grill,4ad4c061f964a520abf720e3,43.665085,-79.384707,Japanese Restaurant,582 Yonge St,M4Y 1Z3,29,1,7.7,35
5,M3B,43.745906,-79.352188,Gonoe Sushi,4b639286f964a5203c842ae3,43.745737,-79.345991,Japanese Restaurant,"1310 Don Mills Rd, Unit 1",M3B 2W6,14,4,6.8,12
6,M5B,43.657162,-79.378937,Hokkaido Ramen Santouka らーめん山頭火,509e9ef6e4b0ab175389a6c5,43.656435,-79.377586,Ramen Restaurant,91 Dundas St E,M5B 1E1,121,2,8.0,304
7,M5B,43.657162,-79.378937,Salad King,4ad4c061f964a52095f720e3,43.657601,-79.381620,Thai Restaurant,340 Yonge St,M5B 1R7,214,2,7.9,484
8,M5B,43.657162,-79.378937,Kinka Izakaya Original,4b2bd898f964a52042bc24e3,43.660596,-79.378891,Japanese Restaurant,398 Church St,M5B 2A2,219,2,8.6,382
9,M5B,43.657162,-79.378937,Kinton Ramen,560b1653498e1a2dfa4af722,43.660632,-79.378880,Ramen Restaurant,396 Church St,M5B 2A2,17,<NA>,8.3,60


In [21]:
# Fix the lat, long of the Neighborhoods
pd.reset_option("max_rows")
fix_lat_long = fsa_df.set_index('PostalCode')[ ['Latitude', 'Longitude'] ]
asian_restaurants = asian_restaurants.join(fix_lat_long, on='Neighborhood')
asian_restaurants = asian_restaurants.drop(columns=[ 'Neighborhood Latitude', 'Neighborhood Longitude'])
asian_restaurants = asian_restaurants.rename(columns={
    'Latitude': 'Neighborhood Latitude',
    'Longitude': 'Neighborhood Longitude'
})
cols = asian_restaurants.columns.to_list()
cols = [cols[0]] + cols[-2:] + cols[1:-2]
asian_restaurants = asian_restaurants[cols]
asian_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Address,Venue Postal Code,Venue Tip Count,Venue Price Tier,Venue Rating,Venue Likes
0,M5A,43.654260,-79.360636,Izumi,50e88fc6e4b007fcbb57aae8,43.649970,-79.360153,Asian Restaurant,NaN,M5A 3C4,4,2,7.4,18
1,M6A,43.718518,-79.464763,Lac Vien Vietnamese Restaurant,4ccc5aebee23a14370591ea8,43.721259,-79.468472,Vietnamese Restaurant,141 Cartwright Ave,M6A 2X3,12,2,7.6,29
2,M4Y,43.665860,-79.383160,Tokyo Sushi,4afca145f964a5208e2422e3,43.665885,-79.386977,Sushi Restaurant,33 St. Joseph St.,M4Y 1J8,24,2,8.1,32
3,M4Y,43.665860,-79.383160,Sushi Garden,4ba6b463f964a520416a39e3,43.664858,-79.384941,Sushi Restaurant,3 Wellesley St W,M4Y 1E7,9,2,7.7,9
4,M4Y,43.665860,-79.383160,Tokyo Grill,4ad4c061f964a520abf720e3,43.665085,-79.384707,Japanese Restaurant,582 Yonge St,M4Y 1Z3,29,1,7.7,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,M4Y,43.665860,-79.383160,Tokyo Kitchen,4ae787b9f964a52020ac21e3,43.668783,-79.385153,Japanese Restaurant,Charles St.,M4Y 1Z8,21,2,7.4,20
141,M4Y,43.665860,-79.383160,Onnki Donburi,59c54d4f2d2fd97564d4cfc8,43.669757,-79.384574,Japanese Restaurant,40 Hayden Street,M4Y 1V8,3,2,7.7,13
142,M6H,43.669005,-79.442259,Miss Korea,527428a5498e8e23580dbe15,43.668950,-79.386165,Korean Restaurant,687 Yonge St,M6H 2C9,17,2,7.5,32
143,M4Y,43.665860,-79.383160,Kokoni Izakaya,4c531b60a724e21e029e3af4,43.664181,-79.380258,Japanese Restaurant,459 Church St,M4Y 2C5,9,2,6.6,1


In [22]:
# Check for the dupes
asian_restaurants.loc[asian_restaurants.duplicated(subset='Venue ID', keep=False)].sort_values(by='Venue ID')

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Address,Venue Postal Code,Venue Tip Count,Venue Price Tier,Venue Rating,Venue Likes
134,M4Y,43.665860,-79.383160,Tokyo Grill,4ad4c061f964a520abf720e3,43.665085,-79.384707,Japanese Restaurant,582 Yonge St,M4Y 1Z3,29,1,7.7,35
4,M4Y,43.665860,-79.383160,Tokyo Grill,4ad4c061f964a520abf720e3,43.665085,-79.384707,Japanese Restaurant,582 Yonge St,M4Y 1Z3,29,1,7.7,35
121,M5J,43.640816,-79.381752,Ki Modern Japanese + Bar,4ae73054f964a5203ca921e3,43.647223,-79.379374,Japanese Restaurant,181 Bay St,M5J 2T3,61,3,8.0,94
114,M5J,43.640816,-79.381752,Ki Modern Japanese + Bar,4ae73054f964a5203ca921e3,43.647223,-79.379374,Japanese Restaurant,181 Bay St,M5J 2T3,61,3,8.0,94
69,M5J,43.640816,-79.381752,Ki Modern Japanese + Bar,4ae73054f964a5203ca921e3,43.647223,-79.379374,Japanese Restaurant,181 Bay St,M5J 2T3,61,3,8.0,94
60,M5J,43.640816,-79.381752,Ki Modern Japanese + Bar,4ae73054f964a5203ca921e3,43.647223,-79.379374,Japanese Restaurant,181 Bay St,M5J 2T3,61,3,8.0,94
126,M5H,43.650571,-79.384568,Fune Japanese Restaurant,4afa1f33f964a520e91622e3,43.648514,-79.386457,Japanese Restaurant,100 Simcoe St.,M5H,27,2,7.7,53
38,M5H,43.650571,-79.384568,Fune Japanese Restaurant,4afa1f33f964a520e91622e3,43.648514,-79.386457,Japanese Restaurant,100 Simcoe St.,M5H,27,2,7.7,53
2,M4Y,43.665860,-79.383160,Tokyo Sushi,4afca145f964a5208e2422e3,43.665885,-79.386977,Sushi Restaurant,33 St. Joseph St.,M4Y 1J8,24,2,8.1,32
136,M4Y,43.665860,-79.383160,Tokyo Sushi,4afca145f964a5208e2422e3,43.665885,-79.386977,Sushi Restaurant,33 St. Joseph St.,M4Y 1J8,24,2,8.1,32


In [23]:
# Final cleaned dataframe
asian_restaurants.drop_duplicates(ignore_index=True, inplace=True)
asian_restaurants

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue ID,Venue Latitude,Venue Longitude,Venue Category,Venue Address,Venue Postal Code,Venue Tip Count,Venue Price Tier,Venue Rating,Venue Likes
0,M5A,43.654260,-79.360636,Izumi,50e88fc6e4b007fcbb57aae8,43.649970,-79.360153,Asian Restaurant,NaN,M5A 3C4,4,2,7.4,18
1,M6A,43.718518,-79.464763,Lac Vien Vietnamese Restaurant,4ccc5aebee23a14370591ea8,43.721259,-79.468472,Vietnamese Restaurant,141 Cartwright Ave,M6A 2X3,12,2,7.6,29
2,M4Y,43.665860,-79.383160,Tokyo Sushi,4afca145f964a5208e2422e3,43.665885,-79.386977,Sushi Restaurant,33 St. Joseph St.,M4Y 1J8,24,2,8.1,32
3,M4Y,43.665860,-79.383160,Sushi Garden,4ba6b463f964a520416a39e3,43.664858,-79.384941,Sushi Restaurant,3 Wellesley St W,M4Y 1E7,9,2,7.7,9
4,M4Y,43.665860,-79.383160,Tokyo Grill,4ad4c061f964a520abf720e3,43.665085,-79.384707,Japanese Restaurant,582 Yonge St,M4Y 1Z3,29,1,7.7,35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,M4Y,43.665860,-79.383160,Tokyo Kitchen,4ae787b9f964a52020ac21e3,43.668783,-79.385153,Japanese Restaurant,Charles St.,M4Y 1Z8,21,2,7.4,20
115,M4Y,43.665860,-79.383160,Onnki Donburi,59c54d4f2d2fd97564d4cfc8,43.669757,-79.384574,Japanese Restaurant,40 Hayden Street,M4Y 1V8,3,2,7.7,13
116,M6H,43.669005,-79.442259,Miss Korea,527428a5498e8e23580dbe15,43.668950,-79.386165,Korean Restaurant,687 Yonge St,M6H 2C9,17,2,7.5,32
117,M4Y,43.665860,-79.383160,Kokoni Izakaya,4c531b60a724e21e029e3af4,43.664181,-79.380258,Japanese Restaurant,459 Church St,M4Y 2C5,9,2,6.6,1


In [24]:
# Save the data to file
with open('asian_restaurants.pkl', 'wb') as f:
        pickle.dump(asian_restaurants, f, pickle.HIGHEST_PROTOCOL)